# Advanced Examples

In [1]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': 'tight', 'dpi': 110}
%load_ext autoreload
%autoreload 2
import logging, warnings
logging.getLogger("pymc").setLevel(logging.FATAL)
warnings.filterwarnings("ignore")

## PyMC

The [Example](example.html) page introduces how to use *muse-inference* for a problem defined with PyMC. Here we consider a more complex problem to highlight additional features. In particular:

* We can estimate any number of parameters with any shapes. Here we have a 2-dimensional array $\mu$ and a scalar $\theta$. Note that by default, *muse-inference* considers any variables which do not depend on others as "parameters" (i.e. the "leaves" of the probabilistic graph). However, the algorithm is not limited to such parameters, and any choice can be selected by providing a list of `params` to the `PyMCMuseProblem` constructor.

* We can work with distributions with limited domain support. For example, below we use the $\rm Beta$ distribution with support on $(0,1)$ and the $\rm LogNormal$ distribution with support on $(0,\infty)$. All necessary transformations are handled internally.

* The data and latent space can include any number of variables, with any shapes. Below we demonstrate an $x$ and $z$ which are 2-dimensional arrays. 

First, load the relevant packages:

In [2]:
%pylab inline
import pymc as pm
from muse_inference.pymc import PyMCMuseProblem

Populating the interactive namespace from numpy and matplotlib


Then define the problem,

In [3]:
def gen_funnel(x=None, σ=None, μ=None, rng=None):
    with pm.Model() as model:
        μ = pm.Beta("μ", 2, 5, size=2) if μ is None else μ
        σ = pm.Normal("σ", 0, 3) if σ is None else σ
        z = pm.LogNormal("z", μ, np.exp(σ/2), size=(100, 2))
        x = pm.Normal("x", z, 1, observed=x)
    return model

generate the model and some data, given some chosen true values of parameters,

In [4]:
θ_true = dict(μ=[0.3, 0.7], σ=1)
with gen_funnel(rng=RandomState(0), **θ_true):
    x_obs = pm.sample_prior_predictive(1, random_seed=0).prior.x[0,0]
model = gen_funnel(x=x_obs)
prob = PyMCMuseProblem(model)

and finally, run MUSE:

In [5]:
θ_start = dict(μ=[0.5, 0.5], σ=0)
result = prob.solve(θ_start=θ_start, progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   0%|          | 6/5050 [00:00<01:24, 59.47it/s]

MUSE:   0%|          | 14/5050 [00:00<01:17, 65.40it/s]

MUSE:   0%|          | 21/5050 [00:00<01:16, 66.03it/s]

MUSE:   1%|          | 29/5050 [00:00<01:12, 69.18it/s]

...

get_H:  53%|█████▎    | 37/70 [00:00<00:00, 67.86it/s]

get_H:  64%|██████▍   | 45/70 [00:00<00:00, 71.13it/s]

get_H:  79%|███████▊  | 55/70 [00:00<00:00, 76.93it/s]

get_H:  91%|█████████▏| 64/70 [00:00<00:00, 69.76it/s]

get_H: 100%|██████████| 70/70 [00:00<00:00, 70.64it/s]

When there are multiple parameters, the starting guess should be specified as as a dictionary, as above.

The parameter estimate is returned as a dictionary,

In [6]:
result.θ

{'μ': array([0.40205548, 0.38608271]), 'σ': array(0.94459269)}

 and the covariance as matrix, with parameters concatenated in the order they appear in the model (or in the order specified in `params`, if that was used):

In [7]:
result.Σ

array([[ 0.02853583,  0.00510861, -0.00817584],
       [ 0.00510861,  0.02903142, -0.00493029],
       [-0.00817584, -0.00493029,  0.0140522 ]])

The `result.ravel` and `result.unravel` functions can be used to convert between dictionary and vector representations of the parameters. For example, to compute the standard deviation for each parameter (the square root of the diagonal of the covariance):

In [8]:
result.unravel(np.sqrt(np.diag(result.Σ)))

{'μ': array([0.16892551, 0.1703861 ]), 'σ': array(0.11854197)}

or to convert the mean parameters to a vector:

In [9]:
result.ravel(result.θ)

array([0.40205548, 0.38608271, 0.94459269])

## Jax

We can also use [Jax](https://jax.readthedocs.io/) to define the problem. In this case we will write out function to generate forward samples and to compute the posterior, and Jax will provide necessary gradients for free. To use Jax, load the necessary packages:

In [10]:
from functools import partial
import jax
import jax.numpy as jnp
from muse_inference.jax import JittableJaxMuseProblem, JaxMuseProblem

Let's implement the noisy funnel problem from the [Example](example.html) page. To do so, extend either `JaxMuseProblem`, or, if your code is able to be JIT compiled by Jax, extend `JittableJaxMuseProblem` and decorate the functions with `jax.jit`:

In [11]:
class JaxFunnelMuseProblem(JittableJaxMuseProblem):

    def __init__(self, N):
        super().__init__()
        self.N = N

    @partial(jax.jit, static_argnums=0)
    def sample_x_z(self, key, θ):
        keys = jax.random.split(key, 2)
        z = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ/2)
        x = z + jax.random.normal(keys[1], (self.N,))
        return (x, z)

    @partial(jax.jit, static_argnums=0)
    def logLike(self, x, z, θ):
        return -(jnp.sum((x - z)**2) + jnp.sum(z**2) / jnp.exp(θ) + 512*θ) / 2

    @partial(jax.jit, static_argnums=0)
    def logPrior(self, θ):
        return -θ**2 / (2*3**2)

Now generate some simulated data, which we set into `prob.x`. Note also the use of `PRNGKey` (rather than `RandomState` for PyMC/Numpy) for random number generation. 

In [12]:
prob = JaxFunnelMuseProblem(10000)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, 0)
prob.x = x

And finally, run MUSE:

In [13]:
prob.solve(θ_start=0., rng=jax.random.PRNGKey(1)) # warmup

In [14]:
result = prob.solve(θ_start=0., rng=jax.random.PRNGKey(1), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   2%|▏         | 102/5050 [00:00<00:06, 797.11it/s]

MUSE:   4%|▍         | 203/5050 [00:00<00:07, 636.47it/s]

MUSE:   6%|▌         | 304/5050 [00:00<00:07, 663.82it/s]

MUSE:   8%|▊         | 390/5050 [00:00<00:06, 719.77it/s]

MUSE: 100%|██████████| 5050/5050 [00:00<00:00, 13842.70it/s]

MUSE: 100%|██████████| 5050/5050 [00:00<00:00, 7532.64it/s] 

get_H:   0%|          | 0/30 [00:00<?, ?it/s]

get_H: 100%|██████████| 30/30 [00:00<00:00, 347.15it/s]

Note that the solution here is obtained around 10X faster that the PyMC version of this in the [Example](example.html) page (the cloud machines which build these docs don't always achieve the 10X, but you see this if you run these examples locally). The Jax interface has much lower overhead, which will be noticeable for very fast posteriors like the one above. 

One convenient aspect of using Jax is that the parameters, `θ`, and latent space, `z`, can be any [pytree](https://jax.readthedocs.io/en/latest/pytrees.html), ie tuples, dictionaries, nested combinations of them, etc... (there is no requirement on the data format of the `x` variable). To demonstrate, consider a problem which is just two copies of the noisy funnel problem:

In [15]:
class JaxPyTreeFunnelMuseProblem(JittableJaxMuseProblem):

    def __init__(self, N):
        super().__init__()
        self.N = N

    @partial(jax.jit, static_argnums=0)
    def sample_x_z(self, key, θ):
        (θ1, θ2) = (θ["θ1"], θ["θ2"])
        keys = jax.random.split(key, 4)
        z1 = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ1/2)
        z2 = jax.random.normal(keys[1], (self.N,)) * jnp.exp(θ2/2)        
        x1 = z1 + jax.random.normal(keys[2], (self.N,))
        x2 = z2 + jax.random.normal(keys[3], (self.N,))        
        return ({"x1":x1, "x2":x2}, {"z1":z1, "z2":z2})

    @partial(jax.jit, static_argnums=0)
    def logLike(self, x, z, θ):
        return (
            -(jnp.sum((x["x1"] - z["z1"])**2) + jnp.sum(z["z1"]**2) / jnp.exp(θ["θ1"]) + 512*θ["θ1"]) / 2
            -(jnp.sum((x["x2"] - z["z2"])**2) + jnp.sum(z["z2"]**2) / jnp.exp(θ["θ2"]) + 512*θ["θ2"]) / 2
        )

    @partial(jax.jit, static_argnums=0)
    def logPrior(self, θ):
        return - θ["θ1"]**2 / (2*3**2) - θ["θ2"]**2 / (2*3**2)

Here, `x`, `θ`, and `z` are all dictionaries. We generate the problem as usual, passing in parameters as dictionaries,

In [16]:
θ_true = dict(θ1=-1., θ2=2.)
θ_start = dict(θ1=0., θ2=0.)

In [17]:
prob = JaxPyTreeFunnelMuseProblem(10000)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, θ_true)
prob.x = x

and run MUSE:

In [18]:
prob.solve(θ_start=θ_start, rng=jax.random.PRNGKey(0)) # warmup

In [19]:
result = prob.solve(θ_start=θ_start, rng=jax.random.PRNGKey(0), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   2%|▏         | 102/5050 [00:00<00:12, 406.19it/s]

MUSE:   3%|▎         | 146/5050 [00:00<00:11, 415.33it/s]

MUSE:   4%|▍         | 190/5050 [00:00<00:11, 421.00it/s]

MUSE:   5%|▍         | 233/5050 [00:00<00:18, 265.07it/s]

...

MUSE: 100%|██████████| 5050/5050 [00:10<00:00, 466.69it/s] 

get_H:   0%|          | 0/50 [00:00<?, ?it/s]

get_H:  32%|███▏      | 16/50 [00:00<00:00, 150.77it/s]

get_H:  68%|██████▊   | 34/50 [00:00<00:00, 167.47it/s]

get_H: 100%|██████████| 50/50 [00:00<00:00, 157.05it/s]

The result is returned as a pytree:

In [20]:
result.θ

{'θ1': DeviceArray(-1.0020632, dtype=float32),
 'θ2': DeviceArray(2.027134, dtype=float32)}

and the covariance as a matrix:

In [21]:
result.Σ

array([[ 3.3823610e-03, -4.6946143e-06],
       [-4.6946157e-06,  2.4767642e-04]], dtype=float32)

The `result.ravel` and `result.unravel` functions can be used to convert between pytree and vector representations of the parameters. For example, to compute the standard deviation for each parameter (the square root of the diagonal of the covariance):

In [22]:
result.unravel(np.sqrt(np.diag(result.Σ)))

{'θ1': DeviceArray(0.05815807, dtype=float32),
 'θ2': DeviceArray(0.01573774, dtype=float32)}

or to convert the mean parameters to a vector:

In [23]:
result.ravel(result.θ)

DeviceArray([-1.0020632,  2.027134 ], dtype=float32)